In [1]:
# Import necessary libraries
import os
from tfsage.download import download_encode
from tfsage.generate import synthesize_experiments
from tfsage import utils

In [2]:
# Create a directory to download the data
os.makedirs("../downloads", exist_ok=True)

In [3]:
# Define the query, target, synthesis experiment IDs and weights
query_id = "ENCFF654RND"
target_id = "ENCFF794LOO"
synthesis_ids = ["ENCFF749OWQ", "ENCFF228QYX", "ENCFF226NCM"]
weights = [0.733717, 0.719159, 0.576055]

In [4]:
# Download the experiments from ENCODE
experiment_ids = [query_id, target_id] + synthesis_ids
for experiment_id in experiment_ids:
    download_encode(experiment_id, f"../downloads/{experiment_id}.bed")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1913  100  1913    0     0   3884      0 --:--:-- --:--:-- --:--:--  3888
100 1756k  100 1756k    0     0  1337k      0  0:00:01  0:00:01 --:--:-- 4132k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1915  100  1915    0     0   4590      0 --:--:-- --:--:-- --:--:--  4592
100  488k  100  488k    0     0   453k      0  0:00:01  0:00:01 --:--:--  453k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1913  100  1913    0     0   3851      0 --:--:-- --:--:-- --:--:--  3849
100  823k  100  823k    0     0   648k      0  0:00:01  0:00:01 --:--:-- 4773k
  % Total    % Received % Xferd  Average Speed   Tim

In [5]:
# Synthesize the data to predict TF binding sites
bed_files = [f"../downloads/{experiment_id}.bed" for experiment_id in synthesis_ids]
predictions = synthesize_experiments(bed_files, weights)
predictions

,chrom,start,end,file_0,file_1,file_2,sum,weight_0,weight_1,weight_2
0,chr1,104794,105184,0,1,0,-0.590613,0.733717,0.719159,0.576055
1,chr1,181549,182019,0,0,1,-0.876821,0.733717,0.719159,0.576055
2,chr1,267860,268196,1,1,1,2.028931,0.733717,0.719159,0.576055
3,chr1,586025,586314,1,1,1,2.028931,0.733717,0.719159,0.576055
4,chr1,778610,779079,1,1,1,2.028931,0.733717,0.719159,0.576055
...,...,...,...,...,...,...,...,...,...,...
55038,chrY,21589078,21589559,1,0,1,0.590613,0.733717,0.719159,0.576055
55039,chrY,24848969,24849399,1,0,0,-0.561497,0.733717,0.719159,0.576055
55040,chrY,26315156,26315414,1,1,1,2.028931,0.733717,0.719159,0.576055
55041,chrY,26549258,26549728,0,0,1,-0.876821,0.733717,0.719159,0.576055


In [6]:
# Generate a test set to evaluate the synthesis results
query_file = f"../downloads/{query_id}.bed"
target_file = f"../downloads/{target_id}.bed"
test_set = utils.generate_test_set(query_file, target_file)
test_set

,chrom,start,end,num,list,positive,negative
0,chr1,181681,181881,1,1,True,False
1,chr1,267916,268116,1,1,True,False
2,chr1,586083,586283,1,1,True,False
3,chr1,778724,778924,1,1,True,False
4,chr1,794715,794915,1,2,False,True
...,...,...,...,...,...,...,...
92927,chrY,20576041,20576241,1,2,False,True
92928,chrY,20581763,20581963,1,2,False,True
92929,chrY,21702595,21702795,1,2,False,True
92930,chrY,21703397,21703597,1,2,False,True


In [7]:
# Sample to control proportion of positive samples in test set
test_set_sampled = utils.stratified_sample(
    test_set, 
    n_samples=1000, 
    p_positive=0.5, 
    random_state=42,
)
test_set_sampled

,chrom,start,end,num,list,positive,negative
0,chr1,8174188,8174388,1,2,False,True
1,chr1,8417837,8418037,1,2,False,True
2,chr1,8463254,8463454,1,2,False,True
3,chr1,10507551,10507751,1,1,True,False
4,chr1,11884383,11884583,1,2,False,True
...,...,...,...,...,...,...,...
995,chrX,130539795,130539995,1,1,True,False
996,chrX,136210619,136210819,1,2,False,True
997,chrX,140766104,140766304,1,1,True,False
998,chrX,148049753,148049953,1,1,True,False


In [8]:
# Proportion of positive samples is controlled at 50%
test_set_sampled["positive"].mean()

0.5

In [9]:
# Evaluate the synthesis results
# `predictions` is expected to have a "score" column
predictions["score"] = predictions["sum"]
result = utils.intersect_predictions_with_test_set(predictions, test_set_sampled)
result

,chrom,start,end,positive,score
0,chr1,8174188,8174388,False,-inf
1,chr1,8417837,8418037,False,-inf
2,chr1,8463254,8463454,False,-inf
3,chr1,10507551,10507751,True,2.028931
4,chr1,11884383,11884583,False,-inf
...,...,...,...,...,...
995,chrX,130539795,130539995,True,2.028931
996,chrX,136210619,136210819,False,-inf
997,chrX,140766104,140766304,True,2.028931
998,chrX,148049753,148049953,True,2.028931


In [10]:
# Sanitize the scores to remove -inf values
result["score"] = utils.sanitize_scores(result["score"])
result

,chrom,start,end,positive,score
0,chr1,8174188,8174388,False,-3.028931
1,chr1,8417837,8418037,False,-3.028931
2,chr1,8463254,8463454,False,-3.028931
3,chr1,10507551,10507751,True,2.028931
4,chr1,11884383,11884583,False,-3.028931
...,...,...,...,...,...
995,chrX,130539795,130539995,True,2.028931
996,chrX,136210619,136210819,False,-3.028931
997,chrX,140766104,140766304,True,2.028931
998,chrX,148049753,148049953,True,2.028931


In [11]:
# Compute suite of binary classification metrics
y_true = result["positive"]
y_score = result["score"]
y_pred = y_score > 0
metrics = utils.compute_classification_metrics(y_true, y_score, y_pred)
metrics

{'roc_auc': 0.9799680000000001,
 'auprc': 0.9667691209890349,
 'f1': 0.9693978282329714,
 'accuracy': 0.969,
 'precision': 0.9571150097465887,
 'recall': 0.982,
 'precision@15%recall': 0.972,
 'precision@20%recall': 0.972,
 'precision@50%recall': 0.972,
 'precision@80%recall': 0.972,
 'precision@90%recall': 0.972,
 'precision@95%recall': 0.972,
 'precision@99%recall': 0.9183673469387755}